In [1]:
%matplotlib inline
import cv2
import numpy as np 
import imutils
import matplotlib.pyplot as plt

In [2]:
cap = cv2.VideoCapture("../data/videos/video.mp4")

In [3]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#Video Writer object
out = cv2.VideoWriter("../data/videos/video_stabilize.mp4",cv2.VideoWriter_fourcc(*"MJPG"),fps,(width*2,height))

In [4]:
ret, prev_frame = cap.read()
prev_frame_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)
old_points = cv2.goodFeaturesToTrack(prev_frame_gray,200,0.01,30)

In [5]:
transforms = np.zeros((n_frames-1,3),dtype = np.float32)
i = 0

In [6]:
while True:
    
    ret , next_frame = cap.read()
    
    if not ret:
        break
    
    next_frame_gray = cv2.cvtColor(next_frame,cv2.COLOR_BGR2GRAY)
    new_points,status,err = cv2.calcOpticalFlowPyrLK(prev_frame_gray,next_frame_gray,old_points,None)
    
    good_old_points = old_points[status == 1]
    good_new_points = new_points[status == 1]
    
    m = cv2.estimateAffinePartial2D(good_old_points,good_new_points)
    
    dx = m[0][0,2]
    dy = m[0][1,2]
    
    da = np.arctan2(m[0][1,0],m[0][0,0])
    transforms[i] = [dx,dy,da]
    i += 1
    old_points =  new_points
    prev_frame_gray = next_frame_gray

In [7]:
trajectory = np.cumsum(transforms,axis = 0)

In [8]:
def moving_average(row,radius):
    conv_filter = np.ones((radius*2 + 1,),dtype = np.float32)/(radius*2 + 1)
    row_padded = np.pad(row,(radius,radius),'edge')
    return_array = np.convolve(row_padded,conv_filter,'same')
    return return_array[radius:-radius]

In [9]:
smooth_trajectory = np.zeros_like(trajectory)
for i in range(3):
    smooth_trajectory[:,i] = moving_average(trajectory[:,i],60)

In [10]:
difference = smooth_trajectory - trajectory
smooth_transforms = transforms + difference

In [11]:
cap = cv2.VideoCapture("../data/videos/video.mp4")

In [12]:
def fix_frame(frame):
    
    M = cv2.getRotationMatrix2D((frame.shape[1]//2,frame.shape[0]//2),0,1.04)
    new_frame = cv2.warpAffine(frame,M,(frame.shape[1],frame.shape[0]))
    return new_frame

In [13]:
ret ,prev_frame = cap.read()

save_frame = np.hstack([prev_frame,prev_frame])
out.write(save_frame)

for i in range(n_frames - 1):
    ret ,next_frame = cap.read()
    if not ret:
        break
        
    m = np.zeros((2,3),dtype = np.float32)
    m[0,0] = np.cos(smooth_transforms[i,2])
    m[0,1] = -np.sin(smooth_transforms[i,2])
    m[1,0] = np.sin(smooth_transforms[i,2])
    m[1,1] = np.cos(smooth_transforms[i,2])
    m[0,2] = smooth_transforms[i,0]
    m[1,2] = smooth_transforms[i,1]
    
    new_frame = cv2.warpAffine(next_frame,m,(next_frame.shape[1],next_frame.shape[0]))
    
    new_frame = fix_frame(new_frame)
    
    save_frame = np.hstack([next_frame,new_frame])
    
    out.write(save_frame)
    
out.release()